In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import torch.autograd as autograd
import os

batchsize=64

class DCGAN_D(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ndf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            #main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            #nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            #main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            #nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output

class DCGAN_G(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ngf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        #main.add_module('initial.{0}.batchnorm'.format(cngf),
                        #nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            #main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            #nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            #main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            #nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output 
###############################################################################

In [2]:
import cv2
import numpy as np
import functions
batchSize = 64

X_train = functions.get_mnist()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

netG = DCGAN_G()
netG.apply(weights_init)
print(netG)

netD = DCGAN_D()
netD.apply(weights_init)
print(netD)

input = torch.FloatTensor(batchSize, 3, 32, 32)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

netD.cuda()
netG.cuda()
input = input.cuda()
one, mone = one.cuda(), mone.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

optimizerD = optim.Adam(netD.parameters(), lr=1e-4, betas=(0.5, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr=1e-4, betas=(0.5, 0.9))

DCGAN_G (
  (main): Sequential (
    (initial.100-256.convt): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial.256.relu): ReLU (inplace)
    (pyramid.256-128.convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.relu): ReLU (inplace)
    (pyramid.128-64.convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.64.relu): ReLU (inplace)
    (final.64-3.convt): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final.3.tanh): Tanh ()
  )
)
DCGAN_D (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (initial.relu.64): LeakyReLU (0.2, inplace)
    (pyramid.64-128.conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.relu): LeakyReLU (0.2, inplace)
    (pyramid.128-256.conv): Conv2

In [ ]:
def calc_gradient_penalty(netD, real_data, fake_data,lamda,batch_size):
    #print real_data.size()
    alpha = torch.rand(batch_size,1,1,1)
    #print (real_data.size())
    alpha = alpha.expand(real_data.size())
    alpha = alpha.cuda()

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    interpolates = interpolates.cuda()
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates).view(batch_size,-1)

    gradients, = autograd.grad(outputs=disc_interpolates.sum(), inputs=interpolates,
                              create_graph=True)
    
    #gradients, = autograd.grad(outputs=disc_interpolates.sum(), inputs=interpolates,
                              #grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                              #create_graph=True, retain_graph=True, only_inputs=True)
    #gradients*gradients
    
    gradient_penalty = ((gradients.view(batch_size,-1).norm(2, dim=1) - 1) ** 2).mean().view(1)* lamda
    return gradient_penalty

In [ ]:
gen_iterations = 0
critic_iters = 5
lamda = 10

for epoch in range(10000):
    data_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): 
            p.requires_grad = True 
        
        for iter_d in range(0,5):
            if i >=len(dataloader):
                continue
            real_cpu, _ = data_iter.next()
            i +=1
            batch_size = real_cpu.size(0)
            real_cpu = real_cpu.cuda()
            input.resize_as_(real_cpu).copy_(real_cpu)
            real_data_v = Variable(input)
            netD.zero_grad()
            netG.zero_grad()
            
            #train with real
            errD_real = netD(real_data_v).mean().view(1)
            errD_real.backward(mone)

            # train with fake
            noise.resize_(batch_size, 100, 1, 1).normal_(0, 1)
            noisev = Variable(noise, volatile = True) # totally freeze netG
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD(fake).mean().view(1)
            errD_fake.backward(one)
            
            # train with gradient penalty
            gradient_penalty = calc_gradient_penalty(netD, real_data_v.data, fake.data,lamda,batch_size)
            gradient_penalty.backward()
            
            D_cost = -errD_real + errD_fake + gradient_penalty
            errD = errD_real - errD_fake 
            optimizerD.step()
            
            
        ############################
        # (2) Update G network
        ###########################
        
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        netG.zero_grad()
        netD.zero_grad()
        
        # in case our last batch was the tail batch of the dataloader,
        # make sure we feed a full batch of noise
        noise.resize_(64, 100, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        errG = netD(fake).mean().view(1)
        errG.backward(mone)
        optimizerG.step()
        gen_iterations += 1


        if gen_iterations % 20 == 0:
            
            print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, 10000, i, len(dataloader), gen_iterations,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
            real_cpu = real_cpu.mul(0.5).add(0.5)
            vutils.save_image(real_cpu, 'WGANGP.png')
            fake = netG(Variable(fixed_noise, volatile=True))
            fake.data = fake.data.mul(0.5).add(0.5)
            vutils.save_image(fake.data, 'WGANGP_fake.png')

[0/10000][100/938][20] Loss_D: 120.555008 Loss_G: 16.957323 Loss_D_real: 137.171539 Loss_D_fake 16.616535
[0/10000][200/938][40] Loss_D: 92.974213 Loss_G: 18.324451 Loss_D_real: 109.842697 Loss_D_fake 16.868484
[0/10000][300/938][60] Loss_D: 95.627014 Loss_G: -15.291439 Loss_D_real: 83.460770 Loss_D_fake -12.166241
[0/10000][400/938][80] Loss_D: 89.099083 Loss_G: -12.100209 Loss_D_real: 80.900665 Loss_D_fake -8.198415
[0/10000][500/938][100] Loss_D: 80.669350 Loss_G: 3.513722 Loss_D_real: 83.960426 Loss_D_fake 3.291076
[0/10000][600/938][120] Loss_D: 69.577332 Loss_G: 9.281697 Loss_D_real: 80.516281 Loss_D_fake 10.938950
[0/10000][700/938][140] Loss_D: 56.077477 Loss_G: 19.412567 Loss_D_real: 75.619743 Loss_D_fake 19.542269
[0/10000][800/938][160] Loss_D: 48.263042 Loss_G: 14.705771 Loss_D_real: 65.616600 Loss_D_fake 17.353558
[0/10000][900/938][180] Loss_D: 48.431168 Loss_G: 14.901585 Loss_D_real: 63.900665 Loss_D_fake 15.469500
[1/10000][60/938][200] Loss_D: 43.496532 Loss_G: 9.87579

[8/10000][480/938][1600] Loss_D: 9.895820 Loss_G: 0.307696 Loss_D_real: 9.840294 Loss_D_fake -0.055526
[8/10000][580/938][1620] Loss_D: 8.071090 Loss_G: -2.597000 Loss_D_real: 7.400770 Loss_D_fake -0.670320
[8/10000][680/938][1640] Loss_D: 8.958690 Loss_G: 0.522664 Loss_D_real: 9.156104 Loss_D_fake 0.197415
[8/10000][780/938][1660] Loss_D: 9.447925 Loss_G: -0.190498 Loss_D_real: 8.742894 Loss_D_fake -0.705030
[8/10000][880/938][1680] Loss_D: 7.344112 Loss_G: -1.364069 Loss_D_real: 7.566219 Loss_D_fake 0.222107
[9/10000][40/938][1700] Loss_D: 10.111905 Loss_G: -2.407081 Loss_D_real: 7.390326 Loss_D_fake -2.721579
[9/10000][140/938][1720] Loss_D: 9.573758 Loss_G: -0.518639 Loss_D_real: 9.451454 Loss_D_fake -0.122304
[9/10000][240/938][1740] Loss_D: 10.090281 Loss_G: 0.428007 Loss_D_real: 10.487998 Loss_D_fake 0.397718
[9/10000][340/938][1760] Loss_D: 9.409363 Loss_G: -0.840538 Loss_D_real: 9.487056 Loss_D_fake 0.077693
[9/10000][440/938][1780] Loss_D: 8.031617 Loss_G: 0.134326 Loss_D_rea

In [ ]:
data_iter.next()

In [ ]:
gradient_penalty

In [ ]:
a = torch.from_numpy(np.array([[1,1],[32,32]],dtype='float32'))
print(a.size())
a.norm(2,dim=1)

In [ ]:
a

In [ ]:
inputv.data

In [ ]:
torch.save(netG.state_dict(), 'netG_try_kmeans_epoch_%d.pth' % (epoch))
torch.save(netD.state_dict(), 'netD_try_kmeans_epoch_%d.pth' % (epoch))

In [ ]:
class DCGAN_D_feature(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ndf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_D_feature, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        #main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        #nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
            
        #output = output.mean(0)
        return output

In [ ]:
import cv2
import numpy as np
batchSize = 1

X_train = functions.get_mnist()
X_train = X_train[0:1000,:,:,:]
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train_ = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train_,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=False, batch_size=batchSize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

In [ ]:
dict_G = torch.load('netG_try_kmeans_epoch_%d.pth' % (1910))
dict_D = torch.load('netD_try_kmeans_epoch_%d.pth' % (1910))
del(dict_D['main.final.256-1.conv.weight'])

feature_dict = {}
netD_feature = DCGAN_D_feature()
netD_feature.load_state_dict(dict_D)
print(netD_feature)
data_iter = iter(dataloader)
for i,data in enumerate(data_iter):
    feature_dict[i] = netD_feature(Variable(data[0])).data.numpy().reshape((1,256*4*4))
    
feature = np.zeros((max(feature_dict.keys()),4096),dtype=np.float32)
for i,data in enumerate(feature):
    data[:] = feature_dict[i] 

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0).fit_predict(feature)

In [ ]:
arg = [np.argwhere(kmeans==n) for n in range(0,10)]
for i in range(0,10):
    cluster = np.take(X_train, arg[i],axis=0).reshape(-1,3,32,32)
    vutils.save_image(torch.from_numpy(cluster), 'cluster%d_%d.png'% (10,i),normalize=True)

In [ ]:
X_train.shape

In [ ]:
np.take(X_train, [0],axis=0).shape

In [ ]:
X_train[6]

In [ ]:
arg = [np.argwhere(kmeans==n) for n in range(0,10)]
pic_list = []
for i,index in enumerate(arg):
    X_train[n,:,:,:]
